In [18]:
import pandas as pd
import os
import multiprocessing as mp
import time
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

In [19]:
index_filepath_eng = "F:/Environmental Baseline Data/Version 4 - Final/Indices/esa_index_vcs_ENG_3aug2021.csv"


In [61]:
df_table_id = df_index_raw.groupby(['Title', 'Data ID'])["Physical and meteorological environment", "Soil and soil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Species at Risk or Species of Special Status and related habitat", "Greenhouse gas (GHG) emissions and climate change", "Air emissions", "Acoustic environment", "Electromagnetism and Corona Discharge", "Human occupancy and resource use", "Heritage resources", "Navigation and navigation safety", "Traditional land and resource use", "Social and cultural well-being", "Human health and aesthetics", "Infrastructure and services", "Employment and economy", "Environmental Obligations", "Rights of Indigenous Peoples"].apply(lambda x : x.astype(int).sum())\
.reset_index().reset_index().rename(columns={'index': 'ID'})
df_table_id.head()

<ipython-input-61-4918b415514d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_table_id = df_index_raw.groupby(['Title', 'Data ID'])["Physical and meteorological environment", "Soil and soil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Species at Risk or Species of Special Status and related habitat", "Greenhouse gas (GHG) emissions and climate change", "Air emissions", "Acoustic environment", "Electromagnetism and Corona Discharge", "Human occupancy and resource use", "Heritage resources", "Navigation and navigation safety", "Traditional land and resource use", "Social and cultural well-being", "Human health and aesthetics", "Infrastructure and services", "Employment and economy", "Environmental Obligations", "Rights of Indigenous Peoples"].apply(lambda x : x.astype(int).sum())\


,ID,Title,Data ID,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,0,TABLE 1 2012 VEGETATION SURVEY LOCATIONS,894928,0,1,18,4,2,27,0,0,0,0,0,0,0,0,2,3,0,0,1,0,0,3
1,1,TABLE 1 ABORIGINAL COMMUNITIES AND THEIR DISTANCE FROM THE PROPOSED PIPELINE RIGHT-OF-WAY,600731,0,0,0,4,4,0,0,0,0,0,0,0,1,0,0,9,2,0,1,0,0,17
2,2,TABLE 1 AREAS ASSESSED UNDER PERMIT 07-368,487965,0,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,TABLE 1 AREAS ASSESSED UNDER PERMIT 2007-252,487871,1,0,1,10,2,0,0,1,0,0,0,0,0,2,7,0,0,0,3,0,0,0
4,4,TABLE 1 BIOSECURITY MANAGEMENT PLAN RESPONSIBILITIES,2545320,0,3,22,0,0,0,0,2,0,0,2,0,1,0,0,0,0,0,6,7,4,0


In [80]:
# =============================== Prepare index dataframe ===============================
df_index_raw = pd.read_csv(index_filepath_eng)

print(len(df_index_raw))
# Create a temporary column in the index dataframe as table identification
df_table_id = df_index_raw.groupby(['Title', 'Data ID'])["Physical and meteorological environment", "Soil and soil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Species at Risk or Species of Special Status and related habitat", "Greenhouse gas (GHG) emissions and climate change", "Air emissions", "Acoustic environment", "Electromagnetism and Corona Discharge", "Human occupancy and resource use", "Heritage resources", "Navigation and navigation safety", "Traditional land and resource use", "Social and cultural well-being", "Human health and aesthetics", "Infrastructure and services", "Employment and economy", "Environmental Obligations", "Rights of Indigenous Peoples"].apply(lambda x : x.astype(int).sum())\
.reset_index().reset_index().rename(columns={'index': 'ID'})
df_index_raw = df_index_raw.merge(df_table_id, left_on=['Title', 'Data ID'], right_on=['Title', 'Data ID'], suffixes=('_x', ''))
df_index_raw.drop(df_index_raw.filter(regex='_x$').columns, axis=1, inplace=True)


28891


<ipython-input-80-67d41ed379b0>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_table_id = df_index_raw.groupby(['Title', 'Data ID'])["Physical and meteorological environment", "Soil and soil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Species at Risk or Species of Special Status and related habitat", "Greenhouse gas (GHG) emissions and climate change", "Air emissions", "Acoustic environment", "Electromagnetism and Corona Discharge", "Human occupancy and resource use", "Heritage resources", "Navigation and navigation safety", "Traditional land and resource use", "Social and cultural well-being", "Human health and aesthetics", "Infrastructure and services", "Employment and economy", "Environmental Obligations", "Rights of Indigenous Peoples"].apply(lambda x : x.astype(int).sum())\


In [81]:
print(len(df_table_id))
print(len(df_index_raw))

15151
28891


In [82]:
df_index_raw.head()

,Unnamed: 0,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,filename,old_filename,ID,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,586,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,14,1,0,0,0,0,26
1,1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,995,32,17,12,327,549,9,82,103,6,1,0,0,0,0,134,153,15,1,11,143,8,153
2,2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-

In [83]:
# Remove bad csvs
df_index = df_index_raw[~df_index_raw['bad_csv']]

In [84]:
assert len(df_index[df_index['bad_csv'] == True]) == 0

In [85]:
# Add a new column - Project Download Path
df_index['Project Download Path'] = df_index['Download folder name']\
    .apply(lambda x: '/projects/{}.zip'.format(x))

<ipython-input-85-f03c43fac486>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_index['Project Download Path'] = df_index['Download folder name']\


In [86]:
# Add a new column - Table Download Path
df_table_filename = df_index.sort_values(['PDF Page Number'])\
    .groupby('ID')['filename'].first().reset_index().rename(columns={'filename': 'Table Name'})
df_table_filename['Table Name'] = df_table_filename['Table Name']\
    .apply(lambda x: x.replace('.csv', '').replace('--', '-'))
df_index = df_index.merge(df_table_filename, left_on='ID', right_on='ID')
df_index['Table Download Path'] = df_index['Table Name']\
    .apply(lambda x: '/tables/{}.zip'.format(x))

In [87]:
df_index.head()

,Unnamed: 0,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,filename,old_filename,ID,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples,Project Download Path,Table Name,Table Download Path
0,0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,586,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,14,1,0,0,0,0,26,/projects/nrthmntn.zip,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2,/tables/nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.zip
1,1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,995,32,17,12,327,549,9,82,103,6,1,0,0,0,0,134,153,15,1,11,143,8,153,/projects/nrthmntn.zip,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2,/tables/nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.zip
2,2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult

In [88]:
# Remove ',All' from Pipeline Location, ESA Section(s) Topics
df_index['Pipeline Location'] = df_index['Pipeline Location']\
    .apply(lambda x: ', '.join([location for location in x.split(', ') if location != 'All']))
df_index['ESA Section(s) Topics'] = df_index['ESA Section(s) Topics']\
    .apply(lambda x: ', '.join([section for section in x.split(', ') if section != 'All']) if type(x) is str else x)

In [89]:
# Update ESA Download URLs
df_index['ESA Folder URL'] = df_index['ESA Folder URL'].apply(lambda x: x.replace('LoadResult', 'View'))

In [90]:
# Prepare a list of column names for the final index files
columns_index = [col for col in df_index.columns.to_list() if col not in (
    'Table Name', 'CSV Download URL', 'Download folder name', 'Zipped Project Link', 'Unnamed: 0',
    'Unnamed: 0.1', 'Index', 'filename', 'old_filename', 'Data ID', 'bad_csv')]

In [91]:
len(df_index)

26665

In [92]:

# =============================== Create Master Index File ===============================
# Added figures back to alpha index file
df_index_with_figure = pd.read_excel('//luxor/data/Branch/Environmental Baseline Data/Version 4 - Final/Indices/ESA_website_ENG.xlsx')
figure_columns = columns_index.copy()
figure_columns.remove('ID')
figure_columns.remove('Project Download Path')
figure_columns.remove('Table Download Path')
df_figure = df_index_with_figure[df_index_with_figure['Content Type'] == 'Figure'][figure_columns]
df_figure['ID'] = df_figure.index + df_index_raw['ID'].max() + 1
df_figure['Application Filing Date'] = df_figure['Application Filing Date'].apply(lambda x: x.strftime('%Y-%m-%d'))


In [101]:
df_figure.head()

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples,ID
0,Figure 13.1-1 EnCana Ekwan Pipeline,Figure,Application to Construct and Operate Ekwan Pipeline,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/268693,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/268706,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/268876,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/293763,"Section 13.1: Introduction, Section 13.1: Project Description, Section 13.3: Assessment Methods,...",1.0,"Land, Air, Vegetation, All",26,107.0,1.41,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15151
1,Figure 13.3-1 CEA Framework,Figure,Application to Construct and Operate Ekwan Pipeline,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/268693,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/268706,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/268876,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/293763,"Section 13.1: Introduction, Section 13.1: Project Description, Section 13.3: Assessment Methods,...",1.0,"Land, Air, Vegetation, All",41,107.0,1.41,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15152
2,Figure 13.4-1 Temperature Normals Measured at the Fort Nelson Airport for the Period 1971 to 2000,Figure,Application to Construct and Operate Ekwan Pipeline,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/268693,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/268706,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/268876,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/293763,"Section 13.1: Introduction, Section 13.1: Project Description, Section 13.3: Assessment Methods,...",1.0,"Land, Air, Vegetation, All",44,107.0,1.41,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15153
3,Figure 13.4-2 Mean Monthly Rainfall and Number of Days with Measurable Rainfall Observed at the ...,Figure,Application to Construct and Operate Ekwan Pipeline,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/268693,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/268706,Large Projects (over 40 km),"Alberta, British Columbia, All",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/268

In [94]:
# Add bad tables
bad_table_columns = columns_index.copy()
bad_table_columns.remove('Project Download Path')
bad_table_columns.remove('Table Download Path')
df_table_bad = df_index_raw[df_index_raw['bad_csv']].sort_values(['ID', 'PDF Page Number']).groupby('ID').first()\
    .reset_index()[bad_table_columns]
df_table_bad['Good Quality'] = False

In [95]:
# Concatenate all tables
df_table = df_index.sort_values(['ID', 'PDF Page Number']).groupby('ID').first()\
    .reset_index()[columns_index]
df_table['Good Quality'] = True

In [96]:
df_index_new = pd.concat([df_figure, df_table, df_table_bad])

# Remove ',All' from Pipeline Location, ESA Section(s) Topics
df_index_new['Pipeline Location'] = df_index_new['Pipeline Location']\
    .apply(lambda x: ', '.join([location for location in x.split(', ') if location != 'All']))
df_index_new['ESA Section(s) Topics'] = df_index_new['ESA Section(s) Topics']\
    .apply(lambda x: ', '.join([section for section in x.split(', ') if section != 'All']) if type(x) is str else x)
# Update ESA Download URls
df_index_new['ESA Folder URL'] = df_index_new['ESA Folder URL'].apply(lambda x: x.replace('LoadResult', 'View'))

In [97]:
len(df_index_new)

24285

In [103]:
df_index_new.tail()

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples,ID,Project Download Path,Table Download Path,Good Quality
899,Table A8-1 Spill Reporting Authorities (cont’d),Table,Application for the Keystone XL Pipeline,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Oil,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/549185,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/549421,Large Projects (over 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Approved,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/550305,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/604643,Appendix A: TransCanada Keystone Environmental\nProtection Plan,33.0,Environment Protection Plan,103,118.0,3.59,No,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,11.0,0.0,0.0,3.0,14311,NaN,NaN,False
900,Table A8-2 Release Reporting Criteria,Table,Application for the Keystone XL Pipeline,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Oil,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/549185,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/549421,Large Projects (over 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Approved,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/550305,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/604643,Appendix A: TransCanada Keystone Environmental\nProtection Plan,33.0,Environment Protection Plan,104,118.0,3.59,No,0.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,14312,NaN,NaN,False
901,Table A9-2 Site Risk Levels,Table,Application for the Keystone XL Pipeline,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Oil,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/549185,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/549421,Large Projects (over 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Approved,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/550305,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/604643,Appendix A: TransCanada Keystone Environmental\nProtection Plan,33.0,Environment Protection Plan,110,118.0,3.59,No,0.0,1.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,14313,NaN,NaN,False
902,Table B-1: Contacts Nelson Jalotjot Project Manager TransCanada PipeLines Limited 450 - 1st Stre...,Table,Application for the Leismer to Kettle River Crossover,Leismer to Kettle River Crossover,7/15/2011,NOVA Gas Transmission Ltd.,Gas,B1-19 - ESA Appendix A Part 1 of 2 - A2A6Q5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/702711,A2A6Q5,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/702730,Large Projects (over 40 km),Alberta,GH-004-2011,Golder Associates,Operating,GC-120,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/704296,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/833910,Appendix A: Environmental Protection Plan,4.0,En

In [98]:
# Create a new folder as the destination for downloading files
new_folder = os.path.join('//luxor/data/Board/ESA_downloads/', 'Jacques_esa_index_files')
if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [99]:
# Export alpha index
df_index_new.to_csv(os.path.join(new_folder, 'ESA_website_ENG.csv'), index=False)

In [107]:
new_folder = os.path.join('//luxor/data/Board/ESA_downloads/', 'download_Bingjie_Mar262021')
new_folder_projects = os.path.join(new_folder, 'projects')
new_folder_tables = os.path.join(new_folder, 'tables')

In [108]:

# Quality check

df_index_new = pd.read_csv(os.path.join(new_folder, 'ESA_website_ENG.csv'))

for project_path in df_index_new['Project Download Path'].unique():
    if type(project_path) is str:
        project_folder = project_path.split('/')[-1]
        if project_folder not in os.listdir(new_folder_projects):
            print('Missing project folder: {}'.format(project_folder))

for table_path in df_index_new['Table Download Path']:
    if type(table_path) is str:
        table_folder = table_path.split('/')[-1]
        if table_folder not in os.listdir(new_folder_tables):
            print('Missing table folder: {}'.format(table_folder))

df_index_new['table missing'] = df_index_new['Table Download Path']\
    .apply(lambda x: x.split('/')[-1] not in os.listdir(new_folder_tables) if type(x) is str else x)

for table_id in df_index_new[df_index_new['table missing'] == True]['ID']:
    print(table_id)
    bundle_for_table(df_index, table_id, new_folder_tables, csv_file_folder, columns_index, readme_table_filepath)

for table_path in df_index_new[df_index_new['table missing'] == True]['Table Download Path']:
    if type(table_path) is str:
        table_folder = table_path.split('/')[-1]
        if table_folder not in os.listdir(new_folder_tables):
            print('Missing table folder: {}'.format(table_folder))


In [ ]:

# Check if certain attributes stay the same for projects
df = pd.read_csv('G:/ESA_downloads/download_Bingjie_Mar262021/ESA_website_ENG.csv')
df_fra = pd.read_csv('G:/ESA_downloads/download_Bingjie_May062021_fra/ESA_website_FRA.csv', encoding='latin-1')


In [ ]:

df_project = df.groupby(['Application Short Name'])[[
    'Application Name',
    'Application Filing Date',
    'Company Name',
    'Commodity',
    'Application Type (NEB Act)',
    'Pipeline Location',
    'Hearing order',
    'Consultant Name',
    'Pipeline Status',
    'Application URL',
    'Project Download Path'
    ]].nunique()

df_project_fra = df_fra.groupby(['Nom abrégé de la demande'])[[
    'Nom de la demande',
    'Dépôt de la demande',
    'Nom de la société',
    'Produit de base',
    'Type de demande (Loi sur l\'Office national de l\'énergie)',
    'Emplacement du pipeline',
    'Ordonnance d\'audience',
    'Nom du consultant',
    'État d\'avancement',
    'URL de la demande',
    "Chemin d'accès pour télécharger le projet"
]].nunique()
